<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gene-Expression-Data-Cleaning" data-toc-modified-id="Gene-Expression-Data-Cleaning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Gene Expression Data Cleaning</a></span></li><li><span><a href="#Clinical-Classification-Target-Extraction" data-toc-modified-id="Clinical-Classification-Target-Extraction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clinical Classification Target Extraction</a></span></li><li><span><a href="#Data-Harmonization-and-Normalization" data-toc-modified-id="Data-Harmonization-and-Normalization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data Harmonization and Normalization</a></span></li></ul></div>

In [16]:
# Packages

import sys
print("Python version: {}". format(sys.version))

import os
print("OS version: {}". format(os.uname()))

import pandas as pd
print("pandas version: {}". format(pd.__version__))

import matplotlib
print("matplotlib version: {}". format(matplotlib.__version__))
import matplotlib.pyplot as plt

import numpy as np
print("NumPy version: {}". format(np.__version__))

import scipy as sp 
print("SciPy version: {}". format(sp.__version__)) 

import sklearn 
print("scikit-learn version: {}". format(sklearn.__version__))

#misc
import random
import time

print('-'*25)

Python version: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
OS version: posix.uname_result(sysname='Darwin', nodename='Bings-MacBook-Pro.local', release='17.4.0', version='Darwin Kernel Version 17.4.0: Sun Dec 17 09:19:54 PST 2017; root:xnu-4570.41.2~1/RELEASE_X86_64', machine='x86_64')
pandas version: 0.23.1
matplotlib version: 2.2.2
NumPy version: 1.14.5
SciPy version: 1.1.0
scikit-learn version: 0.19.1
-------------------------


In [28]:
PATH = "AML-DATA/"

masterGen = pd.read_pickle(PATH + "PD-TARGET-AML-GE")
discoveryClin = pd.read_pickle(PATH + "PD-TARGET-AML-Discovery-ClinicalData")
validationClin = pd.read_pickle(PATH + "PD-TARGET-AML-Validation-ClinicalData")

## Gene Expression Data Cleaning

In [29]:
# No null values
masterGen.isnull().sum().sum()

0

In [30]:
# Duplicate Probes or Genes / Rows or Columns
colCount = masterGen.shape[1]
masterGen = masterGen.loc[:,~masterGen.columns.duplicated()]
print("Duplicate patients removed: " + str(colCount - masterGen.shape[1]))

Duplicate patients removed: 0


In [31]:
# masterGen temporarily transposed for ease of column access
masterGen = masterGen.T

# Duplicate Probes or Genes / Rows or Columns
colCount = masterGen.shape[1]
masterGen = masterGen.loc[:,~masterGen.columns.duplicated()]
print("Duplicate probes removed: " + str(colCount - masterGen.shape[1]))


Duplicate probes removed: 0


In [32]:
# No variance
colCount = masterGen.shape[1]
masterGen.drop(masterGen.std()[(masterGen.std() == 0)].index, axis=1)
print("Probes/Columns with 0 variance removed: " + str(colCount - masterGen.shape[1]))

# master Gen un-transposed
masterGen.T;

Probes/Columns with 0 variance removed: 0


In [33]:
# Check for null/infinite values (should print 'False')
print(np.any(pd.isnull(masterGen.T)))

False


## Clinical Classification Target Extraction

Relapse - recurrence of a past medical condition

Censored - observation period (study) was cut off before the event occurred (relapse occured)


Note: In the future other clinical features (age, race, gender) can be extracted for use in relapse prediction.

Note2: For now, patients not relapsed or censored are dropped

In [34]:
discoveryClin = discoveryClin.T["First Event"]
validationClin = validationClin.T["First Event"]

In [35]:
discoveryClin = discoveryClin[discoveryClin.isin(["Censored", "Relapse"])]
print(discoveryClin.value_counts())

print("-"*30)

validationClin = validationClin[validationClin.isin(["Censored", "Relapse"])]
print(validationClin.value_counts())

Relapse     218
Censored    163
Name: First Event, dtype: int64
------------------------------
Censored    342
Relapse     160
Name: First Event, dtype: int64


In [37]:
from sklearn.preprocessing import LabelBinarizer

labelbinarizer = LabelBinarizer()
discoveryClin = pd.DataFrame(labelbinarizer.fit_transform(discoveryClin), index = discoveryClin.index)
validationClin = pd.DataFrame(labelbinarizer.fit_transform(validationClin), index = validationClin.index)

## Data Harmonization and Normalization